In [ ]:
# v1.0 

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

#Create monthes dict
month_ua = {'січня': 1, 'лютого': 2, 'березня': 3, 'квітня': 4, 'травня': 5, 'червня': 6, 'липня': 7, 'серпня': 8,
 'вересня': 9, 'жовтня': 10, 'листопада': 11, 'грудня': 12}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}

#load the last dataset if it exists
try:
    latest_dataset = pd.read_csv('vacansies_dataset.csv')
    latest_dataset_pd = latest_dataset[['id', 'title', 'time', 'requirements']].set_index('id')
    existing_ids = latest_dataset.id
    print('Number of existing vacancies:', len(existing_ids))
    
except FileNotFoundError:
    print('There is no existing dataset')
    latest_dataset_pd = pd.DataFrame()
    existing_ids = pd.Series()

pages = range(1,50)
vacancies = []
vac_dict = {}

#loop by pages
for p in pages:
    url = 'https://www.work.ua/jobs-data+scientist/?page='+str(p) # url для первой страницы
    page_response = requests.get(url, headers = headers)
    page_content = BeautifulSoup(page_response.content, "html.parser")


    #find all vacancies on page
    items_1 = page_content.find_all('div', {'class': 'card card-hover card-visited wordwrap job-link'})
    items_2 = page_content.find_all('div', {'class': 'card card-hover card-visited wordwrap job-link js-hot-block'})
    items = items_1 + items_2

    #loop by items on each page
    for item in items:
        job_link = item.find('h2').find('a').get('href')
        job_title_full = item.find('h2').find('a').get('title')
        job_title = re.findall(r'(.*)\,\sвакансія\sвід.*', job_title_full)[0]
        ##job_desc = item.find('h2').find('a').text
        #parse job_link to get id
        job_id = re.match(r'\/jobs\/(\d{7})', job_link)
        if job_id:
            job_id = job_id.group(1)
            #Check if the job_id already exist in dataset
            if int(job_id) in existing_ids.values:
                break
        else:
            continue
        #parse job title to get date
        job_date = re.match(r'.*вакансія\sвід\s(\d{1,2})\s(\w*)\s(\d{4})', job_title_full).groups()
        job_date_p = str(month_ua_dict[job_date[1]])+'/'+job_date[0]+'/'+job_date[2]
        

        # get job requirements and save to dictionary
        
        url_job = 'https://www.work.ua/jobs/' + job_id + '/'
        page_response_job = requests.get(url_job, headers = headers)
        page_content_job = BeautifulSoup(page_response_job.content, "html.parser")
        cont = page_content_job.select("div.card.wordwrap ul")
        if len(cont)>1:
            requirements = cont[1].get_text()
            vac_dict.update({job_id:[job_title, job_date, job_date_p, requirements]})

        else:
            cont = page_content_job.select("div.card.wordwrap p")
            requirements = []
            for t in cont[2:]:
                item = t.get_text()
                requirements = requirements + [item]
                vac_dict.update({job_id:[job_title, job_date, job_date_p, requirements]})
                
# Import vacancies dictionary to pandas dataframe    
vac_dict_pr = {'id': list(vac_dict.keys()), 
               'title': [ i[0] for i in list(vac_dict.values()) ], 
               'time': [i[2] for i in list(vac_dict.values())],
              'requirements': [str(i[3]) for i in list(vac_dict.values())]}
vac_df = pd.DataFrame(vac_dict_pr).set_index('id')

print('Number of new vacancies:', len(vac_df))

#Append new data to dataset
new_dataset = vac_df.append(latest_dataset_pd)

# Save pandas df to csv
new_dataset.to_csv('vacansies_dataset.csv')